### 描述符的定义

+ `__get__(self, instance, owner)`: 用于访问属性，返回属性的值
+ `__set__(self, instance, value)`: 将在属性分配操作中调用，不返回任何内容
+ `__delete__(self, instance)`: 控制删除操作，不返回任何内容

以上方法的参数一个不能少，可以使用其他变量名代替，但是个数不能少，返回值的位置固定，建议就用这几个变量做参数

In [39]:
class MyDescriptor:
    def __init__(self):
        print("descriptor working")
        
    def __get__(self, instance, owner):
        print("getting...", self, instance, owner)
        
    def __set__(self, instance, value):
        print("setting...", self, instance, value)
    
    def __delete__(self, instance):
        print("delete...", self, instance)
        
        
class Test(object):
    x = MyDescriptor()
    
    
t = Test()
# t.x
t.x = "X-MAN"
t.x

descriptor working
setting... <__main__.MyDescriptor object at 0x7ff0892d4210> <__main__.Test object at 0x7ff088f73150> X-MAN
getting... <__main__.MyDescriptor object at 0x7ff0892d4210> <__main__.Test object at 0x7ff088f73150> <class '__main__.Test'>


In [40]:
del t.x

delete... <__main__.MyDescriptor object at 0x7ff0892d4210> <__main__.Test object at 0x7ff088f73150>


**小结：**

描述类的：

`__init__(self)`在指派类创建完成时就会执行，因为指派类的创建过程就是指派类的初始化，作为其类属性的描述符实例对象也就完成了创建。

`__get__(self, instance, owner)`在作为指派类的类属性，被指派类的实例对象调用的时候触发，self是描述类实例对象，instance是指派类实例对象，owner是指派类对象。

`__set__(self, instance, value)`在作为指派类的类属性，被指派类的实例对象调用赋值的时候触发，self是描述类实例对象，instance是指派类实例对象，value是赋值的值。

`__delete__(self, instance)`是在删除指派类的类属性时触发。


**** 注意：描述符类本身的实例对象进行属性操作并不会触发三个方法的执行代码。****

#### 作为装饰器使用

In [76]:
class MyDescriptorAsDecoration:
    def __init__(self, func):  # 作为装饰器时，__init__需要设置参数接受被装饰的对象。
        self._func = func
        
    def __get__(self, instance, owner):
        return self._func(instance)  # 必须要return

class TestDecoration:
    @MyDescriptorAsDecoration        
    def test(self):
        return 'hello'
    
t = TestDecoration()
print(t.test)  # 将方法当做属性进行调用

hello


**小结：**

作为装饰器：
    
    利用类装饰器的特性，在装饰之后，随即会调用装饰器类的`__init__`方法对装饰器类进行初始化，实际上被装饰的方法就变成了装饰器类的实例对象，相当于在被装饰的函数的类内部创建了一个以方法名命令的类属性，值是装饰器的实例对象。
    
    以上的整个过程，就相当于描述符类（这里的装饰器类MyDescriptorAsDecoration）和指派类（TestDecoration）之间的关系。
    
    
实际上，MyDescriptorAsDecoration并不能完全作为以装饰器类理解，因为该方式被装饰的函数实际上变成了类属性，只能被当做属性进行调用，不能传递函数的参数（类作为装饰器的时候，内部使用的应该是`__call__`方法），以上的例子只是借用了装饰器的语法糖，使用`@`时，会调用装饰器类的`__init__(self, func)`进行初始化。


实现过程：<font color='red'>依赖于描述符类这个特殊类的性质，依赖其内部的`__get__`、`__set__`、`__delete__`的用法，这是Python这些魔法方法的特性。</font>

*** <del>以上作为类装饰器的用法，实际上也是`@property`的内部实现原理</del> ***

只是类似，实际上内部实现是使用c语言实现，但也使用了`__get__`，我们可以利用该魔法方式手动实现`@property`的效果。

#### 手动实现property效果

In [ ]:
https://www.cnblogs.com/keithtt/p/10223260.html

### 描述类的应用

In [41]:
# 实现摄氏度华氏度的转换

class Celsius:
    '''摄氏度'''
    def __init__(self, value=26.0):
        self.value = float(value)
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = float(value)
        
class Fahrenheit:
    '''华氏度'''
    def __get__(self, instance, owner):
        return instance.cel * 1.8 + 32
    
    def __set__(self, instance, value):
        instance.cel = (float(value) - 32) / 1.8
        
class Temperature:
    cel = Celsius()
    fah = Fahrenheit()
    
temp = Temperature()

# temp.cel = 30
print(temp.fah)
# temp.fah = 100
print(temp.cel)

78.80000000000001
26.0
